In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.utils import plot_model
from keras.layers import Input

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf

df = pd.read_csv("train.csv")

# Separate features and target variable
X = df.drop(columns=['FloodProbability'])
y = df['FloodProbability']

# Define the model as per your architecture
model = Sequential()
model.add(Input(shape=(21,)))
# Input layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Hidden Layer 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Hidden Layer 2
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Hidden Layer 3
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Hidden Layer 4
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(10))  # num_labels should be defined or set to the correct number
# Compile model ONLY ONCE, and with the desired loss/metrics
# Since you are using categorical_crossentropy and to_categorical for target,
# use 'categorical_crossentropy' for loss.
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Impute missing values in X BEFORE scaling
imputer = SimpleImputer(strategy='mean')  # Create an imputer instance
X_imputed = imputer.fit_transform(X)  # Impute missing values in X

# Impute missing values in y (if any)
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1)).ravel() # Impute and reshape back to 1D

# Normalize the imputed features using Min-Max Scaling
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X_imputed) # Scale the imputed data

# Split the normalized data into training and testing sets
# Use y_imputed instead of y
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_imputed, test_size=0.01, random_state=42)

num_batch_size = 32  # Example batch size, adjust as needed
num_epochs = 10

# Convert target variable to categorical using one-hot encoding
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes=10)


# Now fit the model using the encoded target variable
model.fit(X_train, y_train_encoded, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test_encoded))

Epoch 1/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 312s 9ms/step - accuracy: 0.4342 - loss: 15.1705 - val_accuracy: 0.8290 - val_loss: 16.0627
Epoch 2/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 325s 9ms/step - accuracy: 0.5604 - loss: 15.8335 - val_accuracy: 0.1321 - val_loss: 16.0159
Epoch 3/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 319s 9ms/step - accuracy: 0.1556 - loss: 15.8834 - val_accuracy: 0.5826 - val_loss: 16.0309
Epoch 4/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 315s 9ms/step - accuracy: 0.2730 - loss: 15.9137 - val_accuracy: 0.3065 - val_loss: 15.9985
Epoch 5/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 324s 9ms/step - accuracy: 0.2732 - loss: 15.8750 - val_accuracy: 0.1862 - val_loss: 16.0662
Epoch 6/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 302s 9ms/step - accuracy: 0.1652 - loss: 15.8865 - val_accuracy: 0.0700 - val_loss: 16.0588
Epoch 7/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 303s 9ms/step - accuracy: 0.2506 - loss: 15.9524 - val_accuracy: 0.1668 - val_loss: 16.0381
Epoch 8/10
34587/34587 ━━━━━━━━━━━━━━━━━━━━ 325s

In [13]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)

# Print the results
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Test Loss: 16.0875
Test Accuracy: 0.9225
